In [1]:
import requests
import bs4
import string
import re 
import pandas as pd

In [2]:
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
base_url = 'https://can.newonnetflix.info/catalogue/a2z/movies/{}'

In [3]:
Movie_titles = []
Movie_description = []

rng = list('#'+string.ascii_lowercase)

for page in rng:

    if page == '#':
        scrape_url = 'https://can.newonnetflix.info/catalogue/a2z/movies'
    else:
        scrape_url = base_url.format(page)
    
    res = requests.get(scrape_url,headers=agent)
    soup = bs4.BeautifulSoup(res.text,"lxml")
    
    num_of_pages = int(re.findall(r'[0-9]+', soup.select(".wrapper")[0].select(".datemenu")[1].get_text()).pop())
    
    for num in range(1,num_of_pages+1):
        
        scrape_url_page = scrape_url + '?start={}#results'.format(120*(num-1))
        
        res = requests.get(scrape_url_page,headers=agent)
        soup = bs4.BeautifulSoup(res.text,"lxml")
        
        status_movie = True
        
        i = 0

        while status_movie:
            try:
                Movie_titles.append(soup.select(".oldpost")[i].select(".infopop")[1].get_text())
                Movie_description.append(soup.select(".oldpost")[i].select(".infopop")[1]['title'])
                i = i + 1
            except:
                status_movie = False
            

In [10]:
List_of_movies = [re.sub(r"\(\d\d\d\d\)","",Movie_titles[num]) for num in range(len(Movie_titles))]
Desc_of_movies = [re.sub(r"\[.*\]","",Movie_description[num]) for num in range(len(Movie_titles))]

In [15]:
dict = {'Title':List_of_movies,'Desc_of_movies':Desc_of_movies,}
df = pd.DataFrame(dict)
df.head()

,Title,Desc_of_movies
0,10 Cloverfield Lane,"After breaking up with her fiancé, a woman ha..."
1,10 Days in Sun City,After his girlfriend wins the Miss Nigeria pag...
2,10 jours en or,When a carefree bachelor is unexpectedly left ...
3,100 Days Of Solitude (100 días de soledad),Spanish photographer José Díaz spends 100 days...
4,100 Metros,A man who is diagnosed with multiple sclerosis...


In [17]:
df.to_csv('Movie_data.csv',encoding='utf-8-sig')